In [23]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
import time
from IPython.display import clear_output
import urllib.parse

def printProgress(progress, i, missingParts, avgTime, timeRemain, total_time, currPart, errorCount):
    clear_output(wait=True)
    print('Progress: ' + str(progress) + '% || ' +
          'Number of records processed: ' + str(i) + ' || ' +
          'Number of missing parts: ' + str(len(missingParts)) + ' || ' +
          'Estimated time: ' + str(avgTime) + ' seconds || ' +
          'Time elapsed: ' + str(total_time) + ' seconds' + ' || ' +
          'Time remaining: ' + str(timeRemain) + ' seconds || ' +
          'Current part: ' + currPart + ' || ' +
          'Error count: ' + str(errorCount))

# Get parts list
partsDf = pd.read_excel('data\\45292-parts.xlsx', sheet_name='Sheet1')
partsList = partsDf['Prcpart']

# drop duplicates
partsList = list(dict.fromkeys(partsList.tolist()))

# DataFrames
detailsDf = pd.DataFrame() # dataframe to store all info on each part
popDf = pd.DataFrame() # store popularity data
histDf = pd.DataFrame() # store stock and price history
distDf = pd.DataFrame() # store distributors with stock

# keep track of index in df
idx = 0

# keep track of parts that were not found
missingParts = []

# log all errors
log = []

# Used for computing timers and progress trackers
startTime = time.time()
total_time = 0
estimatedTime = []

# Url for searching
url = 'https://www.findchips.com'
search = '/detail/'

# Loop through each part in the part list provided
for i, part in enumerate(partsList):
    try:
        begin_time = time.time()

        # if the part starts with "CON" remove it
        if (part[:3].lower() == 'con') or (part[:3].lower() == 'stk'):
            part = part[3:]

        # convert to url encoding
        partParsed = urllib.parse.quote(part, safe='')

        # get the web page
        page = requests.get(url + search + partParsed)
        soup = BeautifulSoup(page.content, 'html.parser')
        mfg = soup.find('div', class_='analytics-top-distributor')

        # Basically, if there is no mfg, then there is no data.
        # if there is mfg, then we need to see if it is a list or if it is just a single mfg.
        if mfg is None:
            mfgList = []
            missingParts.append(part)
        else:
            mfgList = mfg.find('select', class_='j-select-manufacturer')
            if mfgList is None:
                mfgList = mfg.find('span')
            else:
                mfgList = mfgList.find_all('option')

        # Loop through each mfg for that part
        for mfg in mfgList:

            if isinstance(mfg, bs4.element.Tag) and mfg.has_attr('value'):
                param = mfg['value']
                page = requests.get(url + param)
                soup = BeautifulSoup(page.content, 'html.parser')

            # Analytics top header
            # to do

            # Part info
            results = soup.find_all('li', class_='part-details-list-item')
            if results is not None:
                for item in results:
                    if item is not None:
                        detail = item.find('small').text.strip()
                        value = item.find('p').text.strip()
                        detailsDf.loc[idx, detail] = value

            # CAD models information
            # to do

            # Distributors with stock
            results = soup.find('div', class_='total-inventory-right j-pagination-holder')
            if results is not None:
                results = results.find_all('li', class_='even-odd-list-item')
                distDf.loc[idx, 'part'] = part
                for item in results:
                    count = item.find('span', class_='count').text.strip()
                    distName = item.find('span', class_='distri-name').find('a').text.strip()
                    value = item.find('span', class_='inventory-number').text.strip()
                    distDf.loc[idx, distName] = value

            # Equivalents
            # to do

            # Popularity by part
            results = soup.find_all('div', class_='dash-section col-xs')
            if results is not None:
                for item in results:
                    title = item.find('h4').text.strip()
                    rank = item.find('p', class_='popularity-title').find('strong').text.strip()
                    total = item.find('p', class_='popularity-title').text.strip()
                    total = total[total.index('of') + len('of'):total.index('parts')].strip()
                    change = item.find('span').text.strip()
                    popGraph = item.find('div', class_='popularity-graph-global')                           # not sure how to parse this yet
                    popDf.loc[idx, title] = int(rank.replace(',', ''))
                    popDf.loc[idx, title + ' total'] = int(total.replace(',', ''))

            # The last column has a different class so we must select it seperately
            results = soup.find('div', class_='dash-section col-xs last')
            if results is not None:
                title = results.find('h4').text.strip()
                rank = results.find('p', class_='popularity-title').find('strong').text.strip()
                total = results.find('p', class_='popularity-title').text.strip()
                total = total[total.index('of') + len('of'):total.index('parts')].strip()
                change = results.find('span').text.strip()
                popGraph = results.find('div', class_='popularity-graph-global')                           # not sure how to parse this yet
                popDf.loc[idx, title] = int(rank.replace(',', ''))
                popDf.loc[idx, title + ' total'] = int(total.replace(',', ''))

            # Popularity by region
            # to do

            # Estimated price history
            # to do

            # Estimated stock history
            # to do

            # Market price analysis
            # below code does not work because content seems to be loaded dynamically.
    #         results = soup.find_all('span', class_='j-percent percentage')
    #         if results is not None:
    #             for i, item in enumerate(results):
    #                 percent = item.text.strip()
    #                 if i == 0:
    #                     column = 'price'
    #                 else:
    #                     column = 'stock'
    #                 histDf.loc[idx, title] = float(percent.strip('%'))/100

            # Datasheets
            # to do

            # Related parts
            # to do

            idx += 1

    except Exception as e:
        log.append([part, i, e])

    # calculate progress trackers
    progress = round((i+1)/len(partsList)*100, 2)
    estimatedTime.append(round((len(partsList) * (time.time() - begin_time)), 2))
    avgTime = round((sum(estimatedTime) / len(estimatedTime)), 2)
    total_time = round((total_time + time.time() - begin_time), 2)
    timeRemain = round((avgTime - total_time), 2)
    printProgress(progress, (i+1), missingParts, avgTime, timeRemain, total_time, part, len(log))

# Save
pd.DataFrame(log).to_csv('web_scraped_log2.csv', encoding='utf_8_sig')
pd.DataFrame(missingParts).to_csv('web_scraped_missing_parts2.csv', encoding='utf_8_sig')
detailsDf.to_csv('web_scraped_details2.csv', encoding='utf_8_sig')
popDf.to_csv('web_scraped_popularity2.csv', encoding='utf_8_sig')
histDf.to_csv('web_scraped_history2.csv', encoding='utf_8_sig')
distDf.to_csv('web_scraped_distributors2.csv', encoding='utf_8_sig')

print('Done.')

# Progress: 100.0% || Number of records processed: 11084 || Number of missing parts: 2154 || Estimated time: 77664.13 seconds || Time elapsed: 77666.88 seconds || Time remaining: -2.75 seconds || Current part: ZXS1B3-500 || Error count: 66

Progress: 100.0% || Number of records processed: 11084 || Number of missing parts: 1723 || Estimated time: 55437.05 seconds || Time elapsed: 55435.09 seconds || Time remaining: 1.96 seconds || Current part: ZXS1B3-500 || Error count: 2364
Done.


# Analysis

In [2]:
import pandas as pd
import numpy as np

In [7]:
# logDf = pd.read_csv('web_scraped_log.csv')
infoDf = pd.read_csv('web_scraped_parts_info_2.csv')
# missingDf = pd.read_csv('web_scraped_missing_parts.csv')

In [8]:
infoDf.columns

1567

In [11]:
popCols = [col for col in infoDf.columns if 'Popularity' in col]
popDf = infoDf[popCols]
infoDf = infoDf.drop(popCols, axis=1)
infoDf.to_csv('web_scraped_details.csv', encoding='utf_8_sig')
popDf.to_csv('web_scraped_popularity.csv', encoding='utf_8_sig')

In [4]:
detailsDf = pd.read_csv('web_scraped_details.csv')

In [5]:
detailsDf.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Manufacturer Part Number:', 'Rohs Code:',
       'Part Life Cycle Code:', 'Ihs Manufacturer:', 'Package Description:',
       'Reach Compliance Code:', 'Manufacturer:', 'Risk Rank:',
       ...
       'Input Connector:', 'Output Connector:', 'Output Power-Max:',
       'Number of Signal Lines:', 'Video Standard:', 'Supply Current2-Max:',
       'Supply Voltage2-Nom:', 'Carrier Type:', 'Carrier Type (2):',
       'Carrier Type (3):'],
      dtype='object', length=1056)

In [14]:
detailsDf = detailsDf.replace('', np.nan)
nullCols = detailsDf.isna().sum()
nullCols.sort_values(inplace=True)
print(nullCols)

Unnamed: 0                             0
Unnamed: 0.1                           0
Manufacturer Part Number:           1662
Is Samacsys:                        1662
Part Life Cycle Code:               1662
                                   ...  
Audio Type:                        25177
Peak Primary Current:              25177
Horizontal Pixel Count:            25177
Battery End of Life Indication:    25177
Carrier Type (3):                  25177
Length: 1056, dtype: int64


In [ ]:
popDf = pd.read_csv('output\\web_scraped_popularity.csv')


# Extra code

### This code will get related parts if there is no data on the current search term

In [ ]:
#     while(True):
#         soup = BeautifulSoup(page.content, 'html.parser')
#         mfg = soup.find('div', class_='analytics-top-distributor')

#         # Basically, if there is no mfg, then there is no data.
#         # if there is mfg, then we need to see if it is a list or if it is just a single mfg.
#         if mfg is None:
#             mfgList = []
#         else:
#             mfgList = mfg.find('select', class_='j-select-manufacturer')
#             if mfgList is None:
#                 mfgList = mfg.find('span')
#             else:
#                 mfgList = mfgList.find_all('option')

#         # if mfgList is empty, try one of the related parts
#         if not mfgList:
#             temp = soup.find_all('li', class_='part-details-list-item')
#             if len(temp) > 1:
#                 temp = temp[1].find('a')
#                 page = requests.get(url + temp['href'])
#             else:
#                 break
#         else:
#             break
    
#     if not mfgList:
#         missingParts.append(part)

# Testing

In [20]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
import time
from IPython.display import clear_output
import urllib.parse

def printProgress(progress, i, missingParts, avgTime, timeRemain, total_time, currPart, errorCount):
    clear_output(wait=True)
    print('Progress: ' + str(progress) + '% || ' +
          'Number of records processed: ' + str(i) + ' || ' +
          'Number of missing parts: ' + str(len(missingParts)) + ' || ' +
          'Estimated time: ' + str(avgTime) + ' seconds || ' +
          'Time elapsed: ' + str(total_time) + ' seconds' + ' || ' +
          'Time remaining: ' + str(timeRemain) + ' seconds || ' +
          'Current part: ' + currPart + ' || ' +
          'Error count: ' + str(errorCount))

part = 'CRCW02013K00FKED'
# part = '83619-9011'
# part = '1N4148W-7-F'       # <class 'bs4.element.Tag'>
# part = '83619-9011'      # <class 'bs4.element.NavigableString'>

# DataFrames
detailsDf = pd.DataFrame() # dataframe to store all info on each part
popDf = pd.DataFrame() # store popularity data
histDf = pd.DataFrame() # store stock and price history
distDf = pd.DataFrame() # store distributors with stock

# keep track of index in df
idx = 0

# Url for searching
url = 'https://www.findchips.com'
search = '/detail/'

# if the part starts with "CON" remove it
if (part[:3].lower() == 'con') or (part[:3].lower() == 'stk'):
    part = part[3:]

# convert to url encoding
partParsed = urllib.parse.quote(part, safe='')

# get the web page
page = requests.get(url + search + partParsed)
soup = BeautifulSoup(page.content, 'html.parser')
mfg = soup.find('div', class_='analytics-top-distributor')

# Basically, if there is no mfg, then there is no data.
# if there is mfg, then we need to see if it is a list or if it is just a single mfg.
if mfg is None:
    mfgList = []
    missingParts.append(part)
else:
    mfgList = mfg.find('select', class_='j-select-manufacturer')
    if mfgList is None:
        mfgList = mfg.find('span')
    else:
        mfgList = mfgList.find_all('option')


# Loop through each mfg for that part
for mfg in mfgList:

    if isinstance(mfg, bs4.element.Tag) and mfg.has_attr('value'):
        param = mfg['value']
        page = requests.get(url + param)
        soup = BeautifulSoup(page.content, 'html.parser')

    # Part info
    results = soup.find_all('li', class_='part-details-list-item')
    if results is not None:
        for item in results:
#             print(item)
            detail = item.find('small').text.strip()
            value = item.find('p').text.strip()
            detailsDf.loc[idx, detail] = value

    idx += 1

print('Done.')

Done.
